In [129]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo
from pprint import pprint
from datetime import datetime
import time
import re
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select



In [92]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)


In [93]:
url = "http://www.channelcrawler.com/eng"
browser.visit(url)
browser.is_element_present_by_css("input#queryMinSubs", wait_time=1)
html = browser.html


In [187]:
minSubs = 10000000
driver = webdriver.Chrome()
driver.get(url)
sbox = driver.find_element_by_id('queryMinSubs')
sbox.send_keys(minSubs)
submit = driver.find_element_by_class_name('submitbutton')
submit.click()


In [124]:
select = Select(driver.find_element_by_id('sort-select'))
select.select_by_visible_text('Subscribers (descending)')

In [119]:
html = driver.page_source
channel_soup = bs(html, 'html.parser')
channel_divs = channel_soup.find_all('div', class_="channel")

In [81]:
subs = re.search(r'.+?(?=Subs)', small_text).group(0)
subs = re.sub(r'\t', '', subs).strip()
video_count = re.search(r'.+?(?=Videos)', small_text).group(0) 
video_count = re.sub(r'\t', '', video_count).strip()
join_date = videos = re.search(r'(?<=Join Date:\s).*', small_text).group(0)
join_date = re.sub(r'\t', '', join_date)
join_dto = datetime.strptime(join_date, '%d.%m.%Y')
join_dto

datetime.datetime(2015, 10, 19, 0, 0)

In [188]:
list_10M_subs = []

while next_button:
    html = driver.page_source
    channel_soup = bs(html, 'html.parser')
    channel_divs = channel_soup.find_all('div', class_="channel")
    for channel in channel_divs:
        channel_dict = {}
        title_anchor = channel.find('a')
        try:
            country = channel.find('img')['title']
        except:
            country = ''
        smalls = channel.find_all('small')
        category = smalls[0].get_text()
        small_text = smalls[1].get_text()
        subs = re.search(r'.+?(?=Subs)', small_text).group(0)
        subs = re.sub(r'\t', '', subs).strip()
        video_count = re.search(r'.+?(?=Video)', small_text).group(0) 
        video_count = re.sub(r'\t', '', video_count).strip()
        join_date = videos = re.search(r'(?<=Join Date:\s).*', small_text).group(0)
        join_date = re.sub(r'\t', '', join_date)
        try:
            join_dto = datetime.strptime(join_date, '%d.%m.%Y')
        except:
            join_dto = ''
        channel_dict['title'] = title_anchor['title']
        channel_dict['category'] = category
        channel_dict['channel_url'] = title_anchor['href']
        channel_dict['channel_id'] = re.search(r'(?<=channel/).*', str).group(0)
        channel_dict['subscribers'] = subs
        channel_dict['video_count'] = video_count
        channel_dict['join_date'] = join_date
        channel_dict['country'] = country
        channel_dict['join_dto'] = join_dto
        anchors = channel.find_all('a')
        channel_dict['thumbnail_url'] = anchors[1].find('img')['src']
        list_10M_subs.append(channel_dict)
    try:
        next_button = driver.find_element_by_css_selector("a[rel='next']")
        next_button.click()
    except:
        break
    time.sleep(1)


In [189]:
list_10M_subs

[{'title': 'Maroon 5',
  'category': 'Music',
  'channel_url': 'http://www.youtube.com/channel/UCBVjMGOIkavEAhyqpxJ73Dw',
  'channel_id': 'UCsRM0YB_dabtEPGPTKo-gcw',
  'subscribers': '23,569,942',
  'video_count': '152',
  'join_date': '09.03.2006',
  'country': '',
  'join_dto': datetime.datetime(2006, 3, 9, 0, 0),
  'thumbnail_url': 'https://yt3.ggpht.com/a-/AAuE7mA3aBvEoB6djY9q0xt6P-XI9bNI7xx9gHimug=s88-mo-c-c0xffffffff-rj-k-no'},
 {'title': 'Technical Guruji',
  'category': 'Science & Technology',
  'channel_url': 'http://www.youtube.com/channel/UCOhHO2ICt0ti9KAh-QHvttQ',
  'channel_id': 'UCsRM0YB_dabtEPGPTKo-gcw',
  'subscribers': '11,430,063',
  'video_count': '2095',
  'join_date': '19.10.2015',
  'country': 'India',
  'join_dto': datetime.datetime(2015, 10, 19, 0, 0),
  'thumbnail_url': 'https://yt3.ggpht.com/a-/AAuE7mBrECozW9Jfp_sLjqKFpYtBIEEqZHv9OcLF8A=s88-mo-c-c0xffffffff-rj-k-no'},
 {'title': 'PewDiePie',
  'category': 'Entertainment',
  'channel_url': 'http://www.youtube.c